# Aprendizaje Supervisado con python

# 1. ¿Qué es el aprendizaje automático?

El aprendizaje automático, o _machine learning_, es una aplicación de Inteligencia Artificial (AI) que proporciona a un sistema la habilidad para aprender y mejorar a partir de unos datos sin la necesidad de que esto esté necesariamente incluido en la programación. El ML  is an application of artise centra principalmente en el desarrollo de programas computacionales que puedan leer y usar datos por sí solos.

Ejemplos:
- Sistema que predice si un email va a ser spam o no basándose en una serie de características del mismo
- Sistema de recomendación de artículos a un usuario adaptado a sus gustos
- Estimación del precio de una vivienda basándose en características de la misma
- Agrupación por semejanza de los artículos de la wikipedia
- Detección de comportamientos anómalos en una serie temporal
- etc.

## 1.1 Supervisado vs. no supervisado

El _aprendizaje no supervisado_ trata de descubrir la estructura presente en un conjunto de datos: es decir, trata de buscar semejanzas (o diferencias) entre las observaciones.

Ejemplo: segmentar una base de datos de clientes en grupos por semejanza entre éstos 

El _aprendizaje supervisado_ utiliza _datos etiquetados_, y su objetivo es asignar una etiqueta a las observaciones futuras basándose en las características de las observaciones previas.

Ejemplo: detectar si una transacción realizada con una tarjeta de crédito es fraudulenta o no

## 1.2 Regresión vs. clasificación

Dentro del aprendizaje supervisado, podemos distinguir a su vez dos tipos de problemas según el tipo de variable que estemos prediciendo:
- Clasificación: la variable a predecir es categórica
- Regresión: la variable a predecir es numérica

__Ejemplo:__ ¿Cuál de los siguientes es un problema de clasificación? ¿Y de regresión?
- Señalar si un texto tiene connotación positiva o negativa
- Sistema que predice si un email va a ser spam o no basándose en una serie de características del mismo
- Estimación del precio de una vivienda basándose en carcaterísticas de la misma
- Estimación de la esperanza de vida de un sujeto
- Detección de transacciones fraudulentas realizadas con una tarjeta de crédito

# 2. Fundamentos del aprendizaje automático. Modelos lineales

Vamos a importar las librerías que necesitamos para arrancar

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

E importemos una base de datos sencilla y conocida para los primeros ejemplos. Esto nos ahorrará realizar el análisis exploratorio (ya lo hicimos parcialmente cuando vimos Pandas) e ir directamente al grano.

In [ ]:
casas = pd.read_csv('data/houseprices.csv')

print(casas.head())
print(casas.info())

En este caso no convertiremos las variables categóricas a categorías puesto que scikit-learn, la librería de aprendizaje automático que usaremos, __no las maneja__ En todo caso, no las vamos a necesitar hasta dentro de un rato.

## 2.1 La librería sklearn

Importemos los módulos que necesitamos para entrenar un modelo de regresión. La librería sklearn es tan enorme que lo mejor es importar solo lo necesario para no saturar el entorno

En nuestro caso emplearemos mayormente `linear_model`, un modelo lineal bastante simple para predicción (ajuste de rectas). Hay modelos muchísimo más sofisticados, algunos especialmente eficientes o con aplicaciones específicas, pero describirlos queda algo fuera de nuestro alcance ahora mismo. 

In [ ]:
from sklearn import linear_model

La API de sklearn hace que todos sus modelos (este en concreto es el modelo de regresión lineal) se comporten del mismo lo cual es muy conveniente para aprender a usar la librería, ya que aprendiendo a emplear uno de ellos podremos extrapolar al resto de modelos. Por tanto, si aprendemos a emplear este, no tendremos grandes problemas usando otras cosas como _random forests_, _support vector machines_ o las archiconocidas _redes neuronales_. 

Lo primero que tenemos que definir son los datos que usaremos como variable respuesta y los que usaremos como variables predictoras. 

> Para ambos casos __hay que pasar bien arrays de numpy, bien a dataframes de pandas con columnas numéricas__ 

> `sklearn` solamente maneja variables numericas


En este caso tratamos de predecir `SalePrice` (el precio de venta) usando la variable `GrLivArea` (el área de la casa en pies al cuadrado). Solamente emplearemos una de las muchas variables a nuestra disposición, aunque podríamos hacer modelos más complejos (y mejores) que tengan en cuenta el resto. 

Para saber a qué corresponde cada variable de este dataset, consultar en https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

In [ ]:
### Definimos los arrays de predictores (X) y respuesta (y)
y = casas[['SalePrice']]
X = casas[['GrLivArea']]

Los modelos de scikitlearn son clases. No hemos visto las clases en detalle, pero están ampliamente extendidas y además son muy fáciles de usar. Una clase nos permite crear objetos de un nuevo tipo, y podremos definir nuevas instancias dentro de ese tipo (las cuales pueden tener métodos y atributos propios). Ver las clases en detalle no entra en nuestros objetivos, pero es un aspecto importante que cubrir si quieren continuar su desarrollo en python. 

Para lo que nos es relevante, en este caso, crearemos los modelos invocando al constructor de la clase (`linear_model.LinearRegression()` en este caso): 

In [ ]:
### Inicializamos el modelo (creamos una instancia del modelo para emplearlo)
reg_casas = linear_model.LinearRegression()

Una vez creado, lo iremos modificando e inspeccionando a través de diferentes métodos.

Los modelos se __entrenan__ (se ajustan) con el método `fit(predictores, respuesta)`, que incorpora los datos al modelo. Recordemos que, en general, los modelos de sklearn se tratan **todos igual**, y por tanto con esto tendremos un ejemplo simple que seguir si queremos emplear cualquier otro modelo. 

In [ ]:
### Lo entrenamos con los datos
reg_casas.fit(X, y)

¿Qué hemos obtenido con esto? Investiguemos un poco los métodos que tiene el objeto que acabamos de entrenar.

Podemos obtener los coeficientes del modelo, por ejemplo.

In [ ]:
reg_casas.coef_ # Coeficiente de la regresión lineal (pendiente de la recta)

Para predecir el valor de  nuevas observaciones, usamos el método `.predict()`. Como en este caso no tenemos nuevas observaciones, podemos predecir las que tenemos, que recordemos eran casas['SalePrice'] (¡OJO! ¡Esta práctica de testear el modelo en los datos de train no suele ser una buena idea..!)

In [ ]:
### Lo usamos para predecir
print(reg_casas.predict(X))
print(np.array(casas['SalePrice']))

## 2.1 ¿Qué tal predice mi modelo?

Para hacernos una idea de la precisión de las predicciones, necesitamos alguna función. En regresión se suele usar el _error cuadrático medio_, es decir, la suma de cuadrados de los residuos (la diferencia entre la predicción y la realidad).

O, de una manera equivalente, el $R^2$, que normaliza esta suma de cuadrados con la varianza de los datos y que podemos interpretar como la proporción de la varianza explicada por el modelo.

En sklearn lo podemos calcular con el método `score(predictores, respuesta)` Lo que hace entonces sklearn es comerse la array de predictores, calcular el resultado de aplicarles el modelo, y comparar este resultado con la verdadera respuesta para calcular el $R^2$.

El $R^2$ es una métrica de cuánta varianza de los datos estamos explicando con nuestro modelo. Para un modelo linearl, el $R^2$ queda definido como (1 - [varianza residual / varianza de la variable dependiente]). Por tanto, si un modelo explica perfectamente la varianza de los datos, tendrá un $R^2 = 1$

En este caso, al estar prediciendo los datos que hemos usado para entrenar, estamos calculando el error residual del modelo

In [ ]:
### Obtenemos la estimación del R2
reg_casas.score(X, y)

## Ejercicio: ¡Un primer modelado!

- Leer la base de datos 'data/wages.csv'
- Hacer un rápido análisis exploratorio
- Crear un modelo lineal llamado reg_wage 
- Entrenarlo usando como predictor la columna age y como respuesta earn
- Predecir usando las columnas del dataframe y comparar con las realmente observadas
- Calcular el $R^2$ del modelo

In [ ]:
'''
Rellena el código que falta
'''

## Leer la base de datos 'data/wages.csv' y mostrar el encabezado del dataframe

wages = pd.read_csv(____)

print(wages.head())

## Definimos la variable respuesta (columna earn) y la predictora (columna age)

y = ___
X = ___

## Definir el modelo como hemos hecho más arriba pero usando X_2, y_2 y llamadlo reg_wage

reg_wage = ___

## Entrenar el modelo con .fit(X, y)

___


print('-------\n Predicciones:')
print(reg_wage.predict(X))
print(np.array(casas['SalePrice']))

print('-------\n R2:')
print(reg_wage.score(X, y))

## 2.3 Conjuntos de entrenamiento y test (S)

Para evitar el overfitting tenemos que evaluar el modelo en una serie de datos que no hayamos usado en ningún momento del proceso de ajuste del modelo. 

> Hay que ser muy cuidadosos y aislar totalmente los datos con los que se construye el modelo de los datos con los que vamos a estimar su error

Para hacernos una idea aproximada de cómo va a funcionar cuando le presentemos nuevos datos, tenemos que guardarnos una parte de nuestra base de datos, que __no usaremos para entrenar el modelo__. Sólo la usaremos para predecir, y con lo que obtengamos de ella, calcularemos las métricas de interés.

El subconjunto de datos que vamos a usar para entrenar se llama __conjunto de entrenamiento__ (train set), mientras que el que usamos para evaluarlo se llama __conjunto de test__ (test set)

Aunque podríamos separar los datos nosotros mismos si nos conveniera, podemos hacerlo también con sklearn, puesto que tiene una función integrada preparada para esto precisamente. La función que tenemos que importar es `train_test_split`

In [ ]:
from sklearn.model_selection import train_test_split

X, y = casas[['GrLivArea']], casas[['SalePrice']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

Miremos a ver qué dimensiones tienen los subconjuntos que hemos creado

In [ ]:
print('X train {}'.format(X_train.shape))
print('Y train {}'.format(y_train.shape))
print('X test {}'.format(X_test.shape))
print('Y test {}'.format(y_test.shape))

Repitamos ahora el proceso de entrenamiento y predicción, pero esta vez usando el conjunto de train para entrenar y el de test para calcular el $R^2$.

Vamos a calcular el $R^2$ que obtenemos en el conjunto de test y en el train

In [ ]:
### Instanciamos el modelo y lo entrenamos con los datos de entrenamiento
reg_casas = linear_model.LinearRegression()
reg_casas.fit(X_train, y_train)


print('-------\n R2 test:')

print(reg_casas.score(X_test, y_test))

print('-------\n R2 train:')

print(reg_casas.score(X_train, y_train))

> Obviamente el R2 del conjunto de test es más bajo, pero más cercano a la _realidad_

> Que no haya demasiada diferencia entre ambos es buena señal: no parece que haya **sobreajuste** (overfitting). Pero claro, nos interesará mejorar ese $R^2$, ¡¡y el único modo es usando modelos más complejos!!


![overfitting](figures/underfitting_and_overfitting.png)

## Ejercicio: Predicción básica de salarios 
Estimar usando una partición en train/test el $R^2$ del modelo de los salarios. 
Mostrar el $R^2$ que obtenemos en el conjunto de test y en el train

In [ ]:
'''
Rellena el código que falta
'''
## Leemos la base de datos '../data/wages.csv'
wages = pd.read_csv(____)

## Definimos la variable respuesta (columna earn) y la predictora (columna age)

y = ___
X = ___


### Crear particion en train/test con train_test_split()

X_train, X_test, y_train, y_test = 


### Definimos y entrenamos el modelo
reg_wages = ___
reg_wages.fit(X_train, y_train)

### Visualicemos el R2 del modelo usando los conjuntos de train y test

print('-------\n R2 test:')

print(reg_wages.score(X_test, y_test))

print('-------\n R2 train:')

print(reg_wages.score(X_train, y_train))

## 2.4 Validación cruzada

Obviamente, realizar esta operación una sóla vez puede darnos un resultado demasiado azaroso, sea esto para bien o para mal. Por ello deberíamos repetir este proceso varias veces y calcular su media, para así tener una mejor estimación de lo que podemos esperar cuando lo probemos con datos reales y no depender de la suerte.

Este proceso se conoce como __validación cruzada__ y está implementado en sklearn. El núcleo de esta idea no es más que tomar los datos y dividirlos en train/test un cierto número de veces, haciendo que cada vez el conjunto de train y de test no sea igual al anterior. 

In [ ]:
from sklearn.model_selection import cross_val_score

La función que usaremos será `cross_val_score(modelo, datos, número de pliegues)`

El número de "pliegues" o "cortes" que hagamos afecta el valor que obtenemos. Un valor en torno a 10 es razonable para la mayor parte de modelos.


![validación cruzada](figures/cv.png)


La función nos devuelve una array con los valores del R2 de cada proceso de entrenamiento/test. Por lo tanto, no hay que dividir previamente en el conjunto de test. Ya lo hace la función `cross_val_score` por nosotros.

In [ ]:
casas = pd.read_csv('data/houseprices.csv')
y = casas[['SalePrice']]
X = casas[['GrLivArea']]
reg_casas = linear_model.LinearRegression()


cv_results = cross_val_score(reg_casas, X, y, cv=10)
cv_results

La función `cross_val_score` es un _wrapper_ sobre la función `.fit`, es decir, una función que tiene anidada otra función, en este caso el `.fit()`, a la cual va llamando después de realizar las sucesivas divisiones en train y test

In [ ]:
print(cv_results.mean())
print(cv_results.std())

##  Ejercicio: Validación cruzada de las predicciones en salarios

Estimar por validación cruzada el $R^2$ del modelo que explica el salario con el nivel educativo del trabajador. Mostrar los resultados de la validación, su media y su desviación típica.


In [ ]:
'''
Rellena el código que falta
'''

## Leemos la base de datos 'data/wages.csv'
wages = pd.read_csv(____)

## Definimos la variable respuesta (columna earn) y la predictora (columna age)

y = ___
X = ___


### Crear particion en train/test con train_test_split()

X_train, X_test, y_train, y_test = 


### Definimos el modelo

reg_wages = ___

### Estimar el R2 por validación cruzada en 10 pliegues
cv_results = ___
print(cv_results)

print(cv_results.mean())
print(cv_results.std())

> Observamos cómo, en este caso, al haber realizado antes sólo una validación, los resultados del conjunto de test eran malos _pero por mala suerte_. La estimación realizada por validación cruzada es bastante más fiable.

## 2.5 Más predictores

Podemos incorporar más predictores al modelo pasándolos _como una lista_

`sklearn` __únicamente maneja números__ así que por el momento no vamos a poder usar predictores categóricos en nuestros modelos.

In [ ]:
### Vamos e entrenar el modelo con todas las columnas numéricas

## Leemos los datos
casas = pd.read_csv('data/houseprices.csv')

## Definimos los predictores
predictores = list(casas.select_dtypes(['number']).columns)
predictores.remove('SalePrice')

## Definimos las arrays de predictores y de respuesta
y = casas[['SalePrice']]
X = casas[predictores]

## Definimos el modelo, lo entrenamos y lo evaluamos por cv
reg = linear_model.LinearRegression()
cv_results = cross_val_score(reg, X, y, cv=10)

print(cv_results)
print(np.mean(cv_results))

Podría parecer que mejora el resultado (que lo hace) pero mucho ojo con esto porque

> __Usar el conjunto de test para evaluar los diferentes modelos conduce al overfitting__ 

Para seleccionar un modelo de entre varios, hay que usar otra partición de los datos (**el conjunto de validación**), de forma que no estemos empleando en el entrenamiento información que debe estar reservada para el testing. No vamos a ver este concepto en profundidad, pero es crucial para hacer comparaciones justas entre modelos.

## Ejercicio: Predicciones más complejas de los salarios  

Entrenar un modelo lineal con todos los predictores numéricos de wages 

In [ ]:
'''
Rellena el código que falta
'''

### Vamos e entrenar el modelo con todas las columnas numéricas

## Leemos la base de datos 'data/wages.csv'
wages = pd.read_csv('data/wages.csv')

## Definimos la lista de predictores (todas las columnas numericas )
predictores = list(wages.select_dtypes(['number']).columns)
predictores.remove('earn')

## Definimos la variable respuesta (columna earn) y la predictora (columna age)

y = wages[['earn']]
X = wages[predictores]


## Definimos el modelo,  y lo evaluamos por cv
reg = linear
cv_results = ___

print(cv_results)
print(np.mean(cv_results))

## Principales métodos de sklearn para entrenamiento de los modelos

- linear_model.LinearRegression() (o importar el método en el que estemos interesados)
- modelo.fit(X, y)
- modelo.predict(X)
- modelo.score(X, y)
- model_selection.train_test_split
- model_selection.cross_val_score



# 3. Preprocesado de los datos

Antes de usar un algoritmo de aprendizaje automático, suele ser imprescindible preprocesar los datos para:

- Imputar valores ausentes (NAs)
- Normalización de los datos
- Eliminar atípicos o errores
- Arreglar varianzas desproporcionadas, puntos influyentes o palanca
- Reducir de la dimensión
- Ingeniería de predictores

> IMPORTANTE: Todos estas operaciones de preprocesado hay que realizarlas ÚNICAMENTE considerando el conjunto de entrenamiento, y hacerlas EXACTAMANENTE IGUAL con el conjunto de test. 

De no hacerlo así, estaríamos filtrando información del conjunto de test lo cual _falsearía_ la estimación del error

> ¡¡¡El __overfitting__ está al acecho en todos los pasos del proceso de entrenamiento de un modelo, no sólo en el momento de estimar el error!!!

> - Durante el preprocesado
> - Durante la selección de parámetros
> - Durante la selección del modelo
> - En la estimación del error


## 3.1 Dos observaciones MUY importantes

Aunque `sklearn` admite dataframes como entradas en los modelos y para todas las operaciones de preprocesado como pronto veremos, internamente trabaja __con arrays de numpy__ __y cualquier operación que realice devolverá así mismo una array.__

Esto es fundamental para evitar los errores o para depurar el código. Es un poco incómodo por ser de nivel más bajo que, por ejemplo, `statsmodels`, pero eso se compensa con la flexibilidad que proporciona.

Por otra parte las clases de operaciones de preprocesado de `sklearn` funcionan exactamente como los modelos, algo muy útil como veremos más adelante:

- Primero lo ajustamos a los datos de entrenamiento (con el método `fit`) 
- Y luego usaremos ese objeto  para transformar la columna correspondiente (`transform`), sea la del conjunto de test o la de train

> En una problema real, tendríamos SIEMPRE que ajustar la operación de preprocesado con el conjunto de train y a continuación transformar el conjunto de train y el de test con los parámetros así obtenidos. En caso contrario, estaríamos usando __información del conjunto de test para entrenar el modelo__ y esto produciría overfitting


## 3.2 Imputación de NAs

Los valores ausentes son observaciones a las que les falta el valor de alguna de las variables. Normalmente, los algoritmos de aprendizaje no los pueden procesar y, o fallan o los omiten.

La opción de omitir los NAs siempre está ahí, pero muchas veces suele rentar tratar de _IMPUTAR_ esos valores, es decir, sustituirlos por una estimación razonable del valor que podría tener. Para realizar esto, hay varios métodos:

- Sustituirlos por el valor más común de la distribución (mediana o media para las distribuciones continuas, moda para las discretas)
- Crear una nueva categoría (sólo válido para variables categóricas)
- Entrenar un algoritmo de aprendizaje automático que prediga los valores ausentes

> Hagamos lo que hagamos, es fundamental que las decisiones se tomen sólo considerando el conjunto de entrenamiento, y que la imputación se realice del mismo modo en el conjunto de test



In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd

casas = pd.read_csv('data/houseprices.csv')

In [ ]:
## Examinemos primero si tenemos NAs en el dataframe

nulos = casas.isnull()
suma_nulos = nulos.sum()
ordenados = suma_nulos.sort_values(ascending=False)

print(ordenados)

Si vemos qué significan las distintas columnas del dataset veremos que solamente los valores nulos de Electrical son realmente valores nulos (para `GarageType` y `BsmtQual` simplemente simbolizan que la casa en cuestión no tiene garaje o sótano). Así pués, vamos a arreglar estas columnas con valores nulos "falsos":

In [ ]:
casas.GarageType = casas.GarageType.fillna("NoGarage")
casas.BsmtQual = casas.BsmtQual.fillna("NoBsmt")
nulos = casas.isnull()
suma_nulos = nulos.sum()
ordenados = suma_nulos.sort_values(ascending=False)
print(ordenados)

En la variable Electrical, los NAs son verdaderamente NAs. Para arreglarlos podemos:

- Eliminar las observaciones
- Imputar un valor. Generalmente, la moda
- Crear una nueva categoría
- Hacer un algoritmo de aprendizaje que use el resto de columnas para conjeturar el posible valor (más sobre esto en la segunda parte del curso)

Para eliminar las observaciones a missing, basta con usar el método `dropna`:

In [ ]:
casas_sinNA = casas.dropna()
len(casas), len(casas_sinNA)

Eliminar las observaciones con NAs puede ser una estrategia razonable cuando se trata de un número pequeño de observaciones, pero cuando hay una gran proporción, es más interesante _imputarlas_. Hay varias maneras de imputar, pero hoy vamos a ver sólo las más sencillas: imputación de la mediana para las variables numéricas y de la moda para las categóricas

Las clases de imputadores de sklearn (`Imputer`)  funcionan exactamente como los modelos, algo muy útil como veremos más adelante:

- Primero lo ajustamos a los datos de entrenamiento (con el método `fit`) 
- Y luego lo podemos usar para transformar la columna (`transform`), sea la del conjunto de test o la de train

Por ejemplo, imputemos valores a una columna numérica, y transformémosla:

In [ ]:
## Vamos a crear un Nan en SalePrice, que no tiene ninguno
casas.loc[1220, 'SalePrice'] = np.nan
print(casas['SalePrice'].isnull().sum())

In [ ]:
### Importamos el imputador y lo creamos
# strategy puede ser mean, median o most_frequent
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='median') # , axis=0) # En versiones anteriores necesita el eje

# Lo entrenamos
imp.fit(casas[['SalePrice']])

# Esta línea transforma la columna casas[['SalePrice']] 
casas[['SalePrice']] = imp.transform(casas[['SalePrice']])

##  Comprobemos que ha imputado
print(casas['SalePrice'].isnull().sum())

## Y qué ha imputado
print(casas.loc[1220, 'SalePrice'])
print(casas.SalePrice.median())

Por cierto: la operación de ir preprocesando los datos de una manera secuencial es ir modificando el dataframe o almacenarlo en uno nuevo __es muy poco recomendable__: acabaremos con el entorno lleno de objetos y las posibilidades de errores se disparan.

Afortunadamente luego veremos cóm encapsular estas operaciones en una _tubería_


Para el caso de las categóricas podemos emplear la misma función, aunque hay que prestar atención al método que se selecciona. En este caso tendremos solamente disponibles `most_frequent` y `constant`. 

In [ ]:
## Miramos las distribución de la variable primero
casas.Electrical.value_counts()

In [ ]:
## Creamos el imputador como antes
cat_imp = SimpleImputer(strategy='most_frequent')

## Creamos una copia del dataset para hacer pruebas sin alterar el original
casas_imp = casas.copy()

## Ajustamos el imputador
cat_imp.fit(casas_imp[['Electrical']])

## Hacemos la imputación, sobreescribiendo el dataframe
casas_imp[['Electrical']] = cat_imp.transform(casas_imp[['Electrical']])

## ¿Habrá cambiado la distribución de las variables?
print(casas_imp.Electrical.value_counts())

> Insistimos: lo de alterar el dataframe en cada operación es sólo una conveniencia pedagógica. Trabajar así en la práctica está abocado a la catástrofe!!

## Ejercicio: Imputación de NAs para salarios

Carga la tabla `wages.csv`. Explora la tabla para saber cuántos NAs hay y trata de imputarlos

In [ ]:
'''
Rellena el código que falta
'''

wages = pd.read_csv('data/wages.csv')

## Examinemos primero si tenemos NAs en el dataframe

nulos = ___
suma_nulos = ___
ordenados = ___

print(ordenados)

## Visualicemos la distribución de valores de la columna race con value_counts

print(___)

## Definimos el imputador categórico

cat_imp = ___

## Lo usamos para transformar la columna race

wages.race = ___

## Visualicemos la distribución de valores de la columna race con value_counts

print(___)


## 3.3 Estandarización de datos

La estandarización de nuestro dataset es un requisito preliminar para muchos algoritmos de ML: éstos podrían comportarse mal si las features individuales no parecen más o menos datos distribuidos como una normal estándar ($\mathcal{N}(0,1)$). En la práctica, frecuentemente olvidaremos la forma de la distribución y simplemente transformaremos los datos centrándolos (quitando la media) y dividiendo entre su desviación estándar. 

Por ejemplo, en algunas funciones objetivo el hecho de que alguna columna tenga mucha más varianza que otras puede engañar al algoritmo, de forma que sea incapaz de prestar atención del resto de columnas.

Manos a la obra. Como no tiene sentido calcular media ni std para columnas categóricas, en esta sección solo trabajaremos con las columnas numéricas (más adelante veremos cómo representar las categóricas)

In [ ]:
casas_num = casas._get_numeric_data()

In [ ]:
casas_num.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler().fit(casas_num)

In [ ]:
casas_num_sc = sc.transform(casas_num)

Tip: al hacerlo así, podremos volver a usar `sc.transform` sobre el conjunto de test!

In [ ]:
casas_num.loc[:,:] = casas_num_sc
casas_num.head()

In [ ]:
casas_num.describe()

In [ ]:
casas_num_sc = casas_num.copy()

## Ejercicio: Renormalización en las variables numéricas de salarios

Aplica escalados a las variables numéricas de `wages.csv`

In [ ]:
'''
Rellena el código que falta
'''
## Definimos las columnas numericas

wages_num = ___

wages_num.describe()

## Definamos el normalizador

sc_wages = __

## Usémoslo para transformar todo el dataframe de columnas numéricas

wages_num = ___

wages_num.shape

print(np.mean(wages_num[:,1]))
print(np.std(wages_num[:,1]))



## 3.5 Variables categóricas

Hace algunas secciones atrás hemos visto una forma de representar variables categóricas, asignándolas un número natural a cada valor diferente mediante `LabelEncoder`. Sin embargo, esto no es suficiente ya que los estimadores de sklearn esperan inputs continuos/ordenados.

Por tanto, la representación estándar para variables categóricas es la de one-hot-encoding, fácil de ver con un ejemplo. Supongamos que tenemos una variable que toma tres valores `Rojo`, `Azul`  y `Verde`. Basta pues con crear tres nuevas variables numéricas (las llamadas dummy variabels) y hacer la siguiente transformación (recordad cuando teníamos variables categóricas en la clase de regresión lineal):

* `Rojo` -> (1, 0, 0).
* `Azul` -> (0, 1, 0).
* `Verde` -> (0, 0, 1).

Mediante pandas, basta usar `get_dummies`:

In [ ]:
casas_dum = pd.get_dummies(casas)
casas_dum.head()

In [ ]:
casas_dum.columns.values

De la misma manera, calculamos las dummies de wages

In [ ]:
wages = pd.read_csv("data/wages.csv")
pd.get_dummies(wages).head()

En lugar de usar pandas, también es posible crear las variables dummies directamente a través de sklearn:

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn_pandas import gen_features

Separamos en dos listas los nombres de las variables continuas y en otra las categóricas, puesto que van a ser tratadas de diferente modo.

In [ ]:
predictores_cat = list(casas_imp.select_dtypes(['object']).columns)
predictores_num = list(casas_imp.select_dtypes(['number']).columns)

Definimos por medio de las funciones `gen_features` y `DataFrameMapper` las operaciones a realizar sobre esas columnas

In [ ]:
from sklearn_pandas import gen_features, DataFrameMapper

cat_factory = gen_features(
     columns=predictores_cat,
     classes=[LabelBinarizer]
 )
num_factory = gen_features(
    columns=predictores_num,
    classes=[None]
)
factory = DataFrameMapper(
    cat_factory + num_factory, df_out=True
)

In [ ]:
factory.fit_transform(casas_imp)

# Recapitulando...



Hemos visto como las fases típicas del preprocesado de datos:
1. Imputación de NAs
2. Estandarización
3. Filtrado de NAs
4. Codificación de variables categóricas

Estos pasos se pueden realizar de 1 en 1 "manualmente", pero si se encapsulan de forma que sean compatibles con sklearn, será posible volver a repetir las mismas operaciones sobre nuevos datos. Por ejemplo, volvemos a aplicar el preprocesado a las 100 primeras observaciones de casas:

In [ ]:
casas = pd.read_csv('data/houseprices.csv')
casas_new = casas[:100]

In [ ]:
casas_new_imp = casas_new.copy()

## Hacemos la imputación como antes
cat_imp.fit(casas_new[['Electrical']])
casas_new_imp[['Electrical']] = cat_imp.transform(casas_new[['Electrical']])

In [ ]:
casas_new_sc = casas_new_imp._get_numeric_data().copy()
casas_new_sc.iloc[:,:] = sc.transform(casas_new_sc)

## Principales métodos de sklearn para preprocesado de los datos

- from sklearn.preprocessing import Imputer
- from sklearn_pandas import CategoricalImputer
- from sklearn.preprocessing import StandardScaler
- from sklearn.preprocessing import LabelBinarizer
- from sklearn_pandas import gen_features
- from sklearn_pandas import DataFrameMapper


# 4. Sistematizado el proceso: cañerías (OPCIONAL)

No hemos hecho más que empezar, y ya estamos viendo el nivel de complejidad que puede alcanzar un problema de machine learning, la cantidad de transformaciones que hay que aplicar, transformaciones que hay que cuidarse de hacer sólo en el conjunto de entrenamiento...

Por ejemplo, un proceso típico sería

- Limpieza
- Imputación
- Escalado
- Creación de dummies
- Selección y entrenamiento del modelo
- Optimización  del modelo
- Calcular su error
- __Y todo esto cuidando de dividir repetidas veces el conjunto en entrenamiento y test y de realizar las operaciones en ambos con los mismos parámetros__

Las posibilidades de error o de que haya filtraciones entre uno y otro subconjunto son muy elevadas. Es por ello que se hace necesario sistematizar y blindar todo el proceso.


> Afortunadamente, sklearn pone a nuestra disposición un procedimiento denominado tubería (pipeline) que nos simplifica la vida: sencillamente vamos definiendo las diferentes operaciones que vamos a hacer a los datos de manera simbólica y cómo se enganchan unas con otras, construyendo de esta manera una "cañería" por la que van a "fluir" los datos, que ejecutaremos como si fuera un sólo objeto

In [ ]:
pd.read_csv

In [ ]:
## Ejemplo sencillo de pipeline

## Importamos las funciones que vamos a usar en el modelo
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.model_selection import cross_val_score

## Leemos los datos y definimos los predictores
## En este caso, vamos simplemente a usar los predictores numéricos
casas = pd.read_csv('data/houseprices.csv')
predictores = list(casas.select_dtypes(['number']).columns)
predictores.remove('SalePrice')

## Definimos las arrays de predictores y de respuesta
y = casas[['SalePrice']]
X = casas[predictores]


## Definimos la cañería
steps = [('Reescalado', StandardScaler()),\
         ('Regresion', linear_model.LinearRegression())]

pipeline = Pipeline(steps)


## Ajustamos la cañería a los datos como si fuera un modelo
pipeline.fit(X,y)

## También podemos estimar el error del modelo por validación cruzada
scores = cross_val_score(pipeline, X, y, cv=10)

print(scores)
print(scores.mean())

¡Es así de sencillo! De esta forma tendremos mucho más controlado lo que le hacemos a los datos, asegurándonos de que lo hacemos de forma sistemática y consistente

## 4.1 Ejercicio: Cañerías aplicadas a los salarios 

Con los datos de Wages.csv sin tratar (¡leerlos de nuevo si hace falta!) ajustar una cañería que:

- Prediga con un modelo lineal usando todos los predictores numéricos pero que hayan sido escalados previamente
- Añadir un imputador strategy='most_frequent'



In [ ]:
'''
Rellena el código que falta
'''

from sklearn.impute import SimpleImputer

## Leemos los datos y definimos los predictores
## En este caso, vamos simplemente a usar los predictores numéricos
wages = pd.read_csv('data/wages.csv')
predictores = ___
predictores.remove('earn')

## Dividimos en entrenamiento y test
y = ___
X = ___


## Definimos los pasos
steps = [('Escalado',  ___),\
         ('Regresion', ___)]

## Definimos la cañería
pipeline = ___


## Estimamos el error del modelo por validación cruzada
scores = cross_val_score(___)

print(scores)
print(scores.mean())

## 4.2 ¿Necesita un fontanero?

Las cañerías también pueden _bifurcarse_ de modo que las operaciones sólo afecten a una parte de los datos. Para ello necesitamos las funciones de `sklearn` como `ColumnTransformer`, con el cual podremos operar independientemente en columnas de datos numéricos y categóricos. 

La gran ventaja será que podremos probar diferentes operaciones de preprocesado de una manera muy sencilla

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, LabelBinarizer
from sklearn.model_selection import cross_val_score

import numpy as np
import pandas as pd

Leemos los datos y definimos los predictores, distinguiendo entre numéricos y categóricos

In [ ]:
casas = pd.read_csv('data/houseprices.csv')

predictores_num = casas.select_dtypes(include=['number']).columns
predictores_num = predictores_num.delete(-1)               # Eliminamos "SalePrice", este será nuestro target
predictores_cat = casas.select_dtypes(include=['object']).columns

y = casas.SalePrice                                        # Datos de salida

X = casas[list(predictores_num) + list(predictores_cat)]   # Construimos lo que serán nuestros datos de entrada

Por medio de las funciones convenientes de sklearn, a cada feature categórica, aplicaremos una imputación categórica (que consiste en rellenar NaNs con la moda de la columna), y un LabelBinarizer, que pasa a obtener una representación one-hot binaria (recordad la clase de regresión lineal de la parte de estadística)

Vemos que categorical_features no es más que una lista de tuplas, indicando la columna y las operaciones que se harán a cada columna

In [ ]:
predictores_cat

Definimos las transformaciones que queremos hacer a ambos tipos de variables, tanto las categóricas como las numéricas. En este caso, haremos lo siguiente:
* **Numéricas**: Imputaremos los valores NA con la media del resto de valores y escalaremos los valores (estandarizamos los datos)
* **Categóricas**: Imputamos los NAs con la moda (el valor más frecuente) y pasamos estas variables a formato "one-hot-encoding" para tratarlas en el resto del algoritmo

In [ ]:
transformaciones_numericas = Pipeline(steps=[('imputer', SimpleImputer(missing_values=np.nan, strategy='median') ),
                                      ('scaler',StandardScaler())])
transformaciones_categoricas = Pipeline(steps=[('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))])

Definimos el procesador que tratará las columnas con la función `ColumnTransformer`

In [ ]:
from sklearn.compose import ColumnTransformer
preprocesador = ColumnTransformer(transformers=[('num', transformaciones_numericas, predictores_num),
                                               ('cat', transformaciones_categoricas, predictores_cat)])

Finalmente, añadimos un `SelectKBest` para que haga anova (un método de análisis de la varianza de los datos) y elija las 15 mejores variables, y finalmente una regresión

In [ ]:
steps=[('preprocesador', preprocesador),
       ('anova_filter', SelectKBest(k=15)), 
       ('predictor', linear_model.LinearRegression())]

pipeline = Pipeline(steps)

Finalmente, obtenemos el rendimiento del algoritmo contrastado a través de _cross validation_

In [ ]:
score = cross_val_score(pipeline, X, y, cv= 10)

print(score)
print(np.mean(score))

¡Tenemos una precisión final (en train) del 80%!

También podemos ajustar el modelo a nuestros datos de entrenamiento y posteriormente predecir los de test

In [ ]:
X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X, y, test_size=0.25, random_state=2)
pipeline.fit(X_2_train, y_2_train)
pipeline.score(X_2_train, y_2_train), pipeline.score(X_2_test, y_2_test)

¡En test recibimos una precisión bastante buena también! (~73%)

Pero en definitiva, estas tuberías nos permiten abstraernos y que todo siga su cauce

## Ejercicio: Predicción de salarios usando todo a la vez

Realizar una cañería con la base de datos wages que:

- Impute y convierta a dummies las variables categóricas
- Escale las variables numéricas
- Seleccione los mejores 7 predictores con un filtro anova
- Ajuste un modelo lineal a la variable 'earn'

In [ ]:
'''
Rellena el código que falta
'''

from sklearn.preprocessing import Imputer
## Ejercicio: Una cañería sencillita

## Leemos los datos y definimos los predictores
## En este caso, vamos simplemente a usar los predictores numéricos

wages = pd.read_csv('data/wages.csv')
predictores_cat = ___
predictores_num = ___

# Elimina de predictores_num la columna 'earn', que será nuestro objetivo ahora

## Dividimos en entrenamiento y test
y = ___
X = ___


### Operaciones a realizar a las variables categóricas
cat_factory = gen_features(
     columns = ___,
     classes = [___, ___]
 )

### Concatenarlas con las operaciones a realizar a las variables numéricas
factory = DataFrameMapper(
    cat_factory + 
    [(___, ___)] 
)


## Definimos la cañería
steps = [('feat_prepro', ___), 
         ('anova_filter', ___), 
         ('predictor', ___) ]

pipeline = ___

## Estimamos el error del modelo por validación cruzada
score = cross_val_score(pipeline, X, y, cv= 10)

print(score)
print(np.mean(score))

## Principales métodos de sklearn para control de flujo

- from sklearn.pipeline import Pipeline
- from sklearn_pandas import gen_features
- from sklearn_pandas import DataFrameMapper


# 5 Clasificación

Importemos en un dataframe titanic el archivo titanic.csv y hagamos un análisis exploratorio

In [ ]:
## Leemos los datos
titanic = pd.read_csv('data/titanic_data.csv')

### Exploramos
print(titanic.head())
print(titanic.info())

Observamos lo siguiente:

- Hay 3 columnas que no vamos a usar por tratarse de valores únicos (Name, Ticket, Cabin)
- Descontando esas, nos quedan dos categóricas (Embarked y Sex)


Eliminemos las columnas que no vayamos a usar: Name, Ticket, Cabin

In [ ]:
for columna in ['Name', 'Ticket', 'Cabin']:
    del titanic[columna]
    
titanic.head()    

Vamos a ver los NAs

In [ ]:
nulos = titanic.isnull()
suma_nulos = nulos.sum()
ordenados = suma_nulos.sort_values(ascending=False)

print(ordenados)

Sólo hay nulos en columnas numéricas, así que podremos usar sin mayor problema el imputador de sklearn

Comencemos a definir las operaciones que le vamos a aplicar, que serán:

- Columnas numéricas:
    - Imputar NAs
    - Estandarizar
- Columnas categóricas:
    - Convertir a dummies


In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, LabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn_pandas import gen_features, DataFrameMapper
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

import pandas as pd
import numpy as np


### Leemos el archivo y definimos los predictores

titanic = pd.read_csv('data/titanic_data.csv')

for columna in ['Name', 'Ticket', 'Cabin']:
    del titanic[columna]

titanic.head()

In [ ]:
# Definimos las listas de predictores por tipo
predictores_num = titanic.select_dtypes(['number']).columns
predictores_num = predictores_num.delete(-1)                                 # Este será nuestro target
predictores_cat = titanic.select_dtypes(['object']).columns

print(predictores_num)
print(predictores_cat)

# Definimos los vectores de predictores y la respuesta
y = titanic.Survived
#y = np.ravel(y)
X = titanic[list(predictores_num) + list(predictores_cat)]

In [ ]:
# Hacemos el proceso análogo al de antes

numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(missing_values=np.nan, strategy='median') ),
                                      ('scaler',StandardScaler())])
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocesado = ColumnTransformer(transformers=[('num', numeric_transformer, predictores_num),
                                 ('cat', categorical_transformer, predictores_cat)])

steps=[('preprocessor', preprocesado),
       ('predictor', linear_model.LinearRegression())]

pipe = Pipeline(steps)

In [ ]:
# Estimamos el error del modelo
score = cross_val_score(pipe, X, y, cv=10)
print(score)
print(score.mean())

### Ejercicio:

Con los datos de 'data/spam.csv' crear un algoritmo de clasificación que detecte si un mail es spam o no

In [ ]:
## Leer los datos

## Explorar los datos

## Definir las columnas numéricas y las categóricas

## Definir los predictores y la respuesta

### Operaciones a realizar a las variables categóricas

### Concatenarlas con las operaciones a realizar a las variables numéricas

## Definimos la cañería, o bien realizamos el preprocesado a mano (¡CON MUCHO CUIDADO E INDICÁNDOLO TODO!)

## Estimamos el error del modelo por validación cruzada
